In [ ]:
from PIL import Image


In [ ]:

!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install torch-geometric


Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.5/886.5 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.3 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import torch
from torch_geometric.data import Data
from skimage.segmentation import slic


/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_cluster/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_spline_conv/_version_cuda.so: undefined symbol: _ZN3c1017Registe

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d paultimothymooney/breast-histopathology-images --force
!unzip -oq breast-histopathology-images.zip -d /content/


Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/breast-histopathology-images
License(s): CC0-1.0
 98% 3.03G/3.10G [00:16<00:01, 38.6MB/s]
100% 3.10G/3.10G [00:17<00:00, 195MB/s] 


In [ ]:
!ls /content/IDC_regular_ps50_idx5/0 | head
!ls /content/IDC_regular_ps50_idx5/1 | head


10257
10258
10261
10264
10273
10277
10278
10279
10291
10302
10253
10254
10255
10256
10257
10258
10259
10260
10261
10262


In [ ]:
import glob
png_files = glob.glob("/content/IDC_regular_ps50_idx5/*/*/*.png")
print(f"🖼️ Total PNG files found: {len(png_files)}")
print("Example files:")
print(png_files[:5])


🖼️ Total PNG files found: 277524
Example files:
['/content/IDC_regular_ps50_idx5/10288/1/10288_idx5_x2101_y151_class1.png', '/content/IDC_regular_ps50_idx5/10288/1/10288_idx5_x1901_y301_class1.png', '/content/IDC_regular_ps50_idx5/10288/1/10288_idx5_x2051_y151_class1.png', '/content/IDC_regular_ps50_idx5/10288/1/10288_idx5_x2201_y401_class1.png', '/content/IDC_regular_ps50_idx5/10288/1/10288_idx5_x1951_y251_class1.png']


In [ ]:
!pip install -q torch torchvision torchaudio torch-geometric scikit-image

from PIL import Image
import os
import numpy as np
import torch
from torch_geometric.data import Data
from skimage.segmentation import slic
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from sklearn.metrics import f1_score, cohen_kappa_score, roc_auc_score, accuracy_score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 113.4 MB/s eta 0:00:00


In [ ]:
def create_graph(image_path, label, n_segments=30):

    try:
        image = Image.open(image_path).convert("RGB").resize((100, 100))
        image_np = np.array(image)
        segments = slic(image_np, n_segments=n_segments, compactness=10, start_label=0)
        num_nodes = segments.max() + 1

        features = np.zeros((num_nodes, 3))
        for i in range(num_nodes):
            mask = segments == i
            features[i] = image_np[mask].mean(axis=0) / 255.0

        edges = set()
        for i in range(segments.shape[0] - 1):
            for j in range(segments.shape[1] - 1):
                current = segments[i, j]
                for ni, nj in [(i+1, j), (i, j+1)]:
                    neighbor = segments[ni, nj]
                    if current != neighbor:
                        edges.add((current, neighbor))
                        edges.add((neighbor, current))

        edge_index = torch.tensor(list(edges), dtype=torch.long).t().contiguous()
        x = torch.tensor(features, dtype=torch.float)
        y = torch.tensor([label], dtype=torch.long)
        return Data(x=x, edge_index=edge_index, y=y)
    except Exception as e:
        print(f"❌ Error in {image_path}: {e}")
        return None


In [ ]:
def load_graph_dataset(dataset_dir, limit_per_class=250):
    data = {0: [], 1: []}

    for patient_folder in os.listdir(dataset_dir):
        patient_path = os.path.join(dataset_dir, patient_folder)
        if not os.path.isdir(patient_path):
            continue

        for class_label in ['0', '1']:
            class_path = os.path.join(patient_path, class_label)
            if not os.path.exists(class_path):
                continue

            for fname in os.listdir(class_path):
                if fname.endswith(".png") and len(data[int(class_label)]) < limit_per_class:
                    fpath = os.path.join(class_path, fname)
                    graph = create_graph(fpath, int(class_label))
                    if graph:
                        data[int(class_label)].append(graph)

    return data[0] + data[1]


In [ ]:
dataset_path = "/content/IDC_regular_ps50_idx5"
dataset = load_graph_dataset(dataset_path, limit_per_class=1000)
print(f"✅ Total graphs loaded: {len(dataset)}")


✅ Total graphs loaded: 2000


In [ ]:

train_val, test = train_test_split(dataset, test_size=0.2, stratify=[d.y.item() for d in dataset], random_state=42)
train, val = train_test_split(train_val, test_size=0.25, stratify=[d.y.item() for d in train_val], random_state=42)

train_loader = DataLoader(train, batch_size=32, shuffle=True)
val_loader = DataLoader(val, batch_size=32)
test_loader = DataLoader(test, batch_size=32)

print(f"🧪 Train: {len(train)} | Val: {len(val)} | Test: {len(test)}")


🧪 Train: 1200 | Val: 400 | Test: 400


In [ ]:
from torch.nn import Linear
from torch_geometric.nn import global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        x = self.conv3(x, edge_index).relu()
        x = global_mean_pool(x, batch)
        return self.lin(x)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(in_channels=3, hidden_channels=64, out_channels=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.batch)
        loss = loss_fn(out, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def evaluate(loader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            out = model(batch.x, batch.edge_index, batch.batch)
            pred = out.argmax(dim=1)
            preds.extend(pred.cpu().numpy())
            labels.extend(batch.y.cpu().numpy())
    return preds, labels
for epoch in range(1, 51):
    loss = train()
    val_preds, val_labels = evaluate(val_loader)
    acc = accuracy_score(val_labels, val_preds)
    print(f"📉 Epoch {epoch} | Loss: {loss:.4f} | Val Acc: {acc:.4f}")


📉 Epoch 1 | Loss: 0.6917 | Val Acc: 0.5000
📉 Epoch 2 | Loss: 0.6887 | Val Acc: 0.6925
📉 Epoch 3 | Loss: 0.6816 | Val Acc: 0.5000
📉 Epoch 4 | Loss: 0.6731 | Val Acc: 0.6900
📉 Epoch 5 | Loss: 0.6439 | Val Acc: 0.7075
📉 Epoch 6 | Loss: 0.6092 | Val Acc: 0.7250
📉 Epoch 7 | Loss: 0.5883 | Val Acc: 0.7400
📉 Epoch 8 | Loss: 0.5691 | Val Acc: 0.7475
📉 Epoch 9 | Loss: 0.5670 | Val Acc: 0.7450
📉 Epoch 10 | Loss: 0.5610 | Val Acc: 0.7525
📉 Epoch 11 | Loss: 0.5487 | Val Acc: 0.7600
📉 Epoch 12 | Loss: 0.5393 | Val Acc: 0.7650
📉 Epoch 13 | Loss: 0.5377 | Val Acc: 0.7575
📉 Epoch 14 | Loss: 0.5372 | Val Acc: 0.7575
📉 Epoch 15 | Loss: 0.5355 | Val Acc: 0.7600
📉 Epoch 16 | Loss: 0.5289 | Val Acc: 0.7675
📉 Epoch 17 | Loss: 0.5241 | Val Acc: 0.7625
📉 Epoch 18 | Loss: 0.5230 | Val Acc: 0.7725
📉 Epoch 19 | Loss: 0.5276 | Val Acc: 0.7575
📉 Epoch 20 | Loss: 0.5195 | Val Acc: 0.7475
📉 Epoch 21 | Loss: 0.5114 | Val Acc: 0.7625
📉 Epoch 22 | Loss: 0.5040 | Val Acc: 0.7325
📉 Epoch 23 | Loss: 0.5046 | Val Acc: 0.77

In [ ]:
from sklearn.metrics import (
    accuracy_score, f1_score, cohen_kappa_score, roc_auc_score
)

test_preds, test_labels, test_probs = evaluate_with_probs(test_loader)
print("\n📊 Final Test Metrics:")
print(f"✔️ Test Accuracy       : {accuracy_score(test_labels, test_preds):.4f}")
print(f"✔️ Test Macro F1       : {f1_score(test_labels, test_preds, average='macro'):.4f}")
print(f"✔️ Test Weighted F1    : {f1_score(test_labels, test_preds, average='weighted'):.4f}")
print(f"✔️ Cohen's Kappa       : {cohen_kappa_score(test_labels, test_preds):.4f}")
print(f"✔️ Test AUC (Macro)    : {roc_auc_score(test_labels, test_probs[:, 1]):.4f}")



📊 Final Test Metrics:
✔️ Test Accuracy       : 0.8125
✔️ Test Macro F1       : 0.8125
✔️ Test Weighted F1    : 0.8125
✔️ Cohen's Kappa       : 0.6250
✔️ Test AUC (Macro)    : 0.8988
